# Predicting house prices using k-nearest neighbors regression



In this notebook, you will implement k-nearest neighbors regression. You will:

Find the k-nearest neighbors of a given query input
Predict the output for the query input using the k-nearest neighbors
Choose the best value of k using a validation set

In [1]:
import numpy as np
import pandas as pd

In [2]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

In [3]:
sales = pd.read_csv('kc_house_data_small.csv', dtype = dtype_dict)
train = pd.read_csv('kc_house_data_small_train.csv', dtype = dtype_dict)
test = pd.read_csv('kc_house_data_small_test.csv', dtype = dtype_dict)
validate = pd.read_csv('kc_house_data_validation 2.csv', dtype = dtype_dict)

3. To efficiently compute pairwise distances among data points, we will convert the SFrame (or dataframe) into a 2D Numpy array. First import the numpy library and then copy and paste get_numpy_data() (or equivalent). The function takes a dataset, a list of features (e.g. [‘sqft_living’, ‘bedrooms’]) to be used as inputs, and a name of the output (e.g. ‘price’). It returns a ‘features_matrix’ (2D array) consisting of a column of ones followed by columns containing the values of the input features in the data set in the same order as the input list. It also returns an ‘output_array’, which is an array of the values of the output in the dataset (e.g. ‘price’).



In [4]:
def get_numpy_data(data, features, output):
    data['constant'] = 1 # add a constant column to a dataframe
    # prepend variable 'constant' to the features list
    features = ['constant'] + features
    # select the columns of dataframe given by the ‘features’ list into the SFrame ‘features_sframe’

    # this will convert the features_sframe into a numpy matrix with GraphLab Create >= 1.7!!
    features_matrix = data[features].as_matrix(columns=None)
    # assign the column of data_sframe associated with the target to the variable ‘output_sarray’

    # this will convert the SArray into a numpy array:
    output_array = data[output].as_matrix(columns=None) 
    return(features_matrix, output_array)

Similarly, copy and paste the normalize_features function (or equivalent) from Module 5 (Ridge Regression). Given a feature matrix, each column is divided (element-wise) by its 2-norm. The function returns two items: (i) a feature matrix with normalized columns and (ii) the norms of the original columns.

In [5]:
def normalize_features(features):
    norms = np.sqrt(np.sum(features**2,axis=0))
    normlized_features = features/norms
    return (normlized_features, norms)

Using get_numpy_data (or equivalent), extract numpy arrays of the training, test, and validation sets.

In [6]:
features = [m for m,n in dtype_dict.items() if train[m].dtypes != object]

In [7]:
features

['bathrooms',
 'sqft_living15',
 'sqft_above',
 'grade',
 'yr_built',
 'price',
 'bedrooms',
 'long',
 'sqft_lot15',
 'sqft_living',
 'floors',
 'sqft_lot',
 'waterfront',
 'sqft_basement',
 'yr_renovated',
 'lat',
 'condition',
 'view']

In [8]:
features.remove('price')

In [9]:
training_feature_matrix, training_output = get_numpy_data(train, features, 'price')

In [10]:
testing_feature_matrix, testing_output = get_numpy_data(test, features, 'price')

In [11]:
validating_feature_matrix, validating_output = get_numpy_data(validate, features, 'price')

In computing distances, it is crucial to normalize features. Otherwise, for example, the ‘sqft_living’ feature (typically on the order of thousands) would exert a much larger influence on distance than the ‘bedrooms’ feature (typically on the order of ones). We divide each column of the training feature matrix by its 2-norm, so that the transformed column has unit norm.

IMPORTANT: Make sure to store the norms of the features in the training set. The features in the test and validation sets must be divided by these same norms, so that the training, test, and validation sets are normalized consistently.

e.g. in Python:



In [12]:
features_train, norms = normalize_features(training_feature_matrix)
features_test = testing_feature_matrix / norms
features_valid = validating_feature_matrix / norms

#Compute a single distance

To start, let's just explore computing the “distance” between two given houses. We will take our query house to be the first house of the test set and look at the distance between this house and the 10th house of the training set.

To see the features associated with the query house, print the first row (index 0) of the test feature matrix. You should get an 18-dimensional vector whose components are between 0 and 1. Similarly, print the 10th row (index 9) of the training feature matrix.

In [13]:
print features_test[0]
print features_train[9]

[ 0.01345102  0.01807473  0.01375926  0.01362084  0.01564352  0.01350306
  0.01551285 -0.01346922  0.0016225   0.01759212  0.017059    0.00160518
  0.          0.02481682  0.          0.01345387  0.0116321   0.05102365]
[ 0.01345102  0.00602491  0.01195898  0.0096309   0.01390535  0.01302544
  0.01163464 -0.01346251  0.00156612  0.0083488   0.01279425  0.00050756
  0.          0.          0.          0.01346821  0.01938684  0.        ]


Quiz Question: What is the Euclidean distance between the query house and the 10th house of the training set?

In [14]:
np.sqrt(np.sum((features_train[9] - features_test[0])**2))

0.059723593713980776

Of course, to do nearest neighbor regression, we need to compute the distance between our query house and all houses in the training set.

To visualize this nearest-neighbor search, let's first compute the distance from our query house (features_test[0]) to the first 10 houses of the training set (features_train[0:10]) and then search for the nearest neighbor within this small set of houses. Through restricting ourselves to a small set of houses to begin with, we can visually scan the list of 10 distances to verify that our code for finding the nearest neighbor is working.

Write a loop to compute the Euclidean distance from the query house to each of the first 10 houses in the training set.

Quiz Question: Among the first 10 training houses, which house is the closest to the query house?

In [15]:
distance = {}
for i in range(10):
    distance[i] = np.sqrt(np.sum((features_train[i] - features_test[0])**2))
print distance

{0: 0.060274709162955922, 1: 0.085468811476437465, 2: 0.061499464352793153, 3: 0.053402739792943632, 4: 0.058444840601704413, 5: 0.059879215098128345, 6: 0.0546314049677546, 7: 0.055431083236146074, 8: 0.052383627840220305, 9: 0.059723593713980776}


In [16]:
distance_2 = []
for x,y in distance.items():
    distance_2.append((y,x))

In [17]:
distance_2.sort()

In [18]:
distance_2

[(0.052383627840220305, 8),
 (0.053402739792943632, 3),
 (0.0546314049677546, 6),
 (0.055431083236146074, 7),
 (0.058444840601704413, 4),
 (0.059723593713980776, 9),
 (0.059879215098128345, 5),
 (0.060274709162955922, 0),
 (0.061499464352793153, 2),
 (0.085468811476437465, 1)]

It is computationally inefficient to loop over computing distances to all houses in our training dataset. Fortunately, many of the numpy functions can be vectorized, applying the same operation over multiple values or vectors. We now walk through this process. (The material up to #13 is specific to numpy; if you are using other languages such as R or Matlab, consult relevant manuals on vectorization.)

Consider the following loop that computes the element-wise difference between the features of the query house (features_test[0]) and the first 3 training houses (features_train[0:3]):

In [19]:
for i in xrange(3):
    print features_train[i]-features_test[0]
    # should print 3 vectors of length 18

[  0.00000000e+00  -1.20498190e-02  -5.14364795e-03  -5.50336860e-03
  -3.47633726e-03  -1.63756198e-04  -3.87821276e-03   1.29876855e-05
   6.69281453e-04  -1.05552733e-02  -8.52950206e-03   2.08673616e-04
   0.00000000e+00  -2.48168183e-02   0.00000000e+00  -1.70254220e-05
   0.00000000e+00  -5.10236549e-02]
[  0.00000000e+00  -4.51868214e-03  -2.89330197e-03   1.30705004e-03
  -3.47633726e-03  -1.91048898e-04  -3.87821276e-03   6.16364736e-06
   1.47606982e-03  -2.26610387e-03   0.00000000e+00   7.19763456e-04
   0.00000000e+00  -1.45830788e-02   6.65082271e-02   4.23090220e-05
   0.00000000e+00  -5.10236549e-02]
[  0.00000000e+00  -1.20498190e-02   3.72914476e-03  -8.32384500e-03
  -5.21450589e-03  -3.13866046e-04  -7.75642553e-03   1.56292487e-05
   1.64764925e-03  -1.30002801e-02  -8.52950206e-03   1.60518166e-03
   0.00000000e+00  -2.48168183e-02   0.00000000e+00   4.70885840e-05
   0.00000000e+00  -5.10236549e-02]


In [20]:
print features_train[0:3] - features_test[0]

[[  0.00000000e+00  -1.20498190e-02  -5.14364795e-03  -5.50336860e-03
   -3.47633726e-03  -1.63756198e-04  -3.87821276e-03   1.29876855e-05
    6.69281453e-04  -1.05552733e-02  -8.52950206e-03   2.08673616e-04
    0.00000000e+00  -2.48168183e-02   0.00000000e+00  -1.70254220e-05
    0.00000000e+00  -5.10236549e-02]
 [  0.00000000e+00  -4.51868214e-03  -2.89330197e-03   1.30705004e-03
   -3.47633726e-03  -1.91048898e-04  -3.87821276e-03   6.16364736e-06
    1.47606982e-03  -2.26610387e-03   0.00000000e+00   7.19763456e-04
    0.00000000e+00  -1.45830788e-02   6.65082271e-02   4.23090220e-05
    0.00000000e+00  -5.10236549e-02]
 [  0.00000000e+00  -1.20498190e-02   3.72914476e-03  -8.32384500e-03
   -5.21450589e-03  -3.13866046e-04  -7.75642553e-03   1.56292487e-05
    1.64764925e-03  -1.30002801e-02  -8.52950206e-03   1.60518166e-03
    0.00000000e+00  -2.48168183e-02   0.00000000e+00   4.70885840e-05
    0.00000000e+00  -5.10236549e-02]]


Note that the output of this vectorized operation is identical to that of the loop above, which can be verified below:

In [21]:
# verify that vectorization works
results = features_train[0:3] - features_test[0]
print results[0] - (features_train[0]-features_test[0])
# should print all 0's if results[0] == (features_train[0]-features_test[0])
print results[1] - (features_train[1]-features_test[0])
# should print all 0's if results[1] == (features_train[1]-features_test[0])
print results[2] - (features_train[2]-features_test[0])
# should print all 0's if results[2] == (features_train[2]-features_test[0])

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


# Perform 1-nearest neighbor regression

Now that we have the element-wise differences, it is not too hard to compute the Euclidean distances between our query house and all of the training houses. First, write a single-line expression to define a variable ‘diff’ such that ‘diff[i]’ gives the element-wise difference between the features of the query house and the i-th training house.

To test your code, print diff[-1].sum(), which should be -0.0934339605842.

In [22]:
diff = features_train[:] - features_test[0]
diff

array([[  0.00000000e+00,  -1.20498190e-02,  -5.14364795e-03, ...,
         -1.70254220e-05,   0.00000000e+00,  -5.10236549e-02],
       [  0.00000000e+00,  -4.51868214e-03,  -2.89330197e-03, ...,
          4.23090220e-05,   0.00000000e+00,  -5.10236549e-02],
       [  0.00000000e+00,  -1.20498190e-02,   3.72914476e-03, ...,
          4.70885840e-05,   0.00000000e+00,  -5.10236549e-02],
       ..., 
       [  0.00000000e+00,  -3.01245476e-03,   8.35842791e-04, ...,
         -9.19146535e-06,   0.00000000e+00,  -5.10236549e-02],
       [  0.00000000e+00,  -3.01245476e-03,   2.44323277e-03, ...,
         -1.63183862e-05,   0.00000000e+00,  -5.10236549e-02],
       [  0.00000000e+00,  -3.01245476e-03,  -3.92203156e-03, ...,
          3.61719513e-05,   0.00000000e+00,  -5.10236549e-02]])

In [23]:
features_train - features_test[0]

array([[  0.00000000e+00,  -1.20498190e-02,  -5.14364795e-03, ...,
         -1.70254220e-05,   0.00000000e+00,  -5.10236549e-02],
       [  0.00000000e+00,  -4.51868214e-03,  -2.89330197e-03, ...,
          4.23090220e-05,   0.00000000e+00,  -5.10236549e-02],
       [  0.00000000e+00,  -1.20498190e-02,   3.72914476e-03, ...,
          4.70885840e-05,   0.00000000e+00,  -5.10236549e-02],
       ..., 
       [  0.00000000e+00,  -3.01245476e-03,   8.35842791e-04, ...,
         -9.19146535e-06,   0.00000000e+00,  -5.10236549e-02],
       [  0.00000000e+00,  -3.01245476e-03,   2.44323277e-03, ...,
         -1.63183862e-05,   0.00000000e+00,  -5.10236549e-02],
       [  0.00000000e+00,  -3.01245476e-03,  -3.92203156e-03, ...,
          3.61719513e-05,   0.00000000e+00,  -5.10236549e-02]])

In [24]:
diff[-1].sum()

-0.09343399874654644

The next step in computing the Euclidean distances is to take these feature-by-feature differences in ‘diff’, square each, and take the sum over feature indices. That is, compute the sum of squared feature differences for each training house (row in ‘diff’).

By default, ‘np.sum’ sums up everything in the matrix and returns a single number. To instead sum only over a row or column, we need to specifiy the ‘axis’ parameter described in the np.sum documentation. In particular, ‘axis=1’ computes the sum across each row.



In [25]:
total_row = np.sum(diff**2, axis=1)

In [26]:
total_row.shape

(5527,)

In [27]:
diff.shape

(5527, 18)

computes this sum of squared feature differences for all training houses. Verify that the two expressions


In [28]:
np.sum(diff**2, axis=1)[15]

0.0033070590284564457

In [29]:
np.sum(diff[15]**2)

0.0033070590284564453

With this result in mind, write a single-line expression to compute the Euclidean distances from the query to all the instances. Assign the result to variable distances.

Hint: don't forget to take the square root of the sum of squares.

Hint: distances[100] should contain 0.0237082324496.

In [30]:
np.sqrt(sum(diff[100]**2))

0.023708232416678195

Now you are ready to write a function that computes the distances from a query house to all training houses. The function should take two parameters: (i) the matrix of training features and (ii) the single feature vector associated with the query.

In [31]:
def compute_distances(features_query):
    diff = features_train - features_test[features_query]
    distances = np.sqrt(np.sum(diff**2, axis=1))
    return distances

In [32]:
dist = compute_distances(2)

In [33]:
dist

array([ 0.01954476,  0.06861035,  0.02165079, ...,  0.02433478,
        0.02622734,  0.02637942])

In [34]:
min(dist)

0.0028604955575117085

In [35]:
np.argmin(dist)

382

Quiz Question: What is the predicted value of the query house based on 1-nearest neighbor regression?

In [36]:
training_output[382]

249000.0

#Perform k-nearest neighbor regression

Using the functions above, implement a function that takes in

the value of k;
the feature matrix for the instances; and
the feature of the query
and returns the indices of the k closest training houses. For instance, with 2-nearest neighbor, a return value of [5, 10] would indicate that the 6th and 11th training houses are closest to the query house.

In [37]:
def k_nearest_neighbors(k, feat_query):
    distance = compute_distances(feat_query)
#     print np.sort(distance)[:k]
    return np.argsort(distance)[0:k]

Quiz Question: Take the query house to be third house of the test set (features_test[2]). What are the indices of the 4 training houses closest to the query house?

In [38]:
k_nearest_neighbors(4,2)

array([ 382, 1149, 4087, 3142])

Now that we know how to find the k-nearest neighbors, write a function that predicts the value of a given query house. For simplicity, take the average of the prices of the k nearest neighbors in the training set. The function should have the following parameters:

the value of k;
the feature matrix for the instances;
the output values (prices) of the instances; and
the feature of the query, whose price we’re predicting.
The function should return a predicted value of the query house.

In [42]:
def predict_output_of_query(k, features_train, output_train, features_query):
    prediction = np.sum(output_train[k_nearest_neighbors(k,features_query)])/k
    return prediction

Quiz Question: Make predictions for the first 10 houses in the test set, using k=10. What is the index of the house in this query set that has the lowest predicted value? What is the predicted value of this house?

In [45]:
for m in range(10):
    print m, predict_output_of_query(10, features_train, training_output, m)

0 881300.0
1 431860.0
2 460595.0
3 430200.0
4 766750.0
5 667420.0
6 350032.0
7 512800.7
8 484000.0
9 457235.0


# Choosing the best value of k using a validation set

There remains a question of choosing the value of k to use in making predictions. Here, we use a validation set to choose this value. Write a loop that does the following:

For k in [1, 2, … 15]:

Make predictions for the VALIDATION data using the k-nearest neighbors from the TRAINING data.
Compute the RSS on VALIDATION data
Report which k produced the lowest RSS on validation data.

In [50]:
rss_all = np.zeros(15)
for k in range(1,16):
    predictions_k = predict_output_of_query(k, features_train, training_output, features_valid)
    rss_all[k-1] = np.sum((predictions_k-validating_output)**2)

IndexError: arrays used as indices must be of integer (or boolean) type